# 隔夜收益率

[Overnight Returns and Firm-Specific Investor Sentiment](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010)

> **Abtract**: We explore the possibility that overnight returns can serve as a measure of firm-specific investor sentiment by analyzing whether they exhibit characteristics expected of a sentiment measure. First, we document short-term persistence in overnight returns, consistent with existing evidence of short-term persistence in share demand of sentiment-influenced retail investors. Second, we find that short-term persistence is stronger for harder-to-value firms, consistent with evidence that sentiment plays a larger role when there is less objective data available for valuation. Third, we show that stocks with high (low) overnight returns underperform (outperform) over the longer-term, consistent with evidence of temporary sentiment-driven mispricing.  

> **p 2, I**: The recent work of Berkman, Koch, Tuttle, and Zhang (2012) suggests that a stock’s
 vernight (close-to-open) return can serve as a measure of firm-level sentiment.

> **p 3, I**: Specifically, Berkman et al. (2012) find that attention-generating events (high absolute returns or
 trong net buying by retail investors) on one day lead to higher demand by individual investors,
 oncentrated near the open of the next trading day...This creates temporary price pressure at the
 pen, resulting in elevated overnight returns that are reversed during the trading day.

> **p 3, I**: We conduct three sets of analyses. **In the first
 e test for short-run persistence in overnight returns.** The basis for expecting this from a
 easure of sentiment is the evidence in Barber et al. (2009) that the order imbalances of retail
 nvestors, who are the investors most likely to exhibit sentiment, persist for periods extending
 ver several weeks...In the third analysis we
 xamine whether stocks with high overnight returns underperform those with low overnight
 eturns over the long term.

## 安装软件包

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

### 数据包

In [ ]:
import os
import quiz_helper
from zipline.data import bundles

In [ ]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

### 构建管道引擎

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

### 查看数据

构建管道引擎后，我们获取时段结束时股票池中的股票。我们将使用这些 ticker 生成风险模型的收益率数据。

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

# 获取收益率数据

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## 获取股价数据的辅助函数

In [ ]:
from quiz_helper import get_pricing

## 将股价数据放入 dataframe 中

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## 行业数据辅助函数
我们将为你创建一个对象，它会针对每支股票定义一个行业。行业由整数表示。我们继承了 Classifier 类。[Classifier 文档](https://www.quantopian.com/posts/pipeline-classifiers-are-here)以及 [Classifier 的源代码](https://github.com/quantopian/zipline/blob/master/zipline/pipeline/classifiers/classifier.py)

In [ ]:
from zipline.pipeline.classifiers import Classifier
from zipline.utils.numpy_utils import int64_dtype
class Sector(Classifier):
    dtype = int64_dtype
    window_length = 0
    inputs = ()
    missing_value = -1

    def __init__(self):
        self.data = np.load('../../data/project_4_sector/data.npy')

    def _compute(self, arrays, dates, assets, mask):
        return np.where(
            mask,
            self.data[assets],
            self.missing_value,
        )

In [ ]:
sector = Sector()

## 我们将使用 2 年的数据计算因子

**注意：**2 年前的日期是休市日期。管道软件包不会处理开始或结束日期是休市的日期。为了解决这个问题，我们再往回推 2 天，这时候市场没有休市。

In [ ]:
factor_start_date = universe_end_date - pd.DateOffset(years=2, days=2)
factor_start_date

## 分步介绍”Returns”类

我们将分步介绍 `Returns` 类，因为我们将创建一个继承 `Returns` 的新类，并用该新类计算自定义收益率。

### Returns 继承自 CustomFactor
zipline 软件包具有一个 [zipline.pipeline.factors.Returns](https://www.zipline.io/appendix.html?highlight=returns#zipline.pipeline.factors.Returns) 类，它继承自  [zipline.pipeline.CustomFactor](https://www.zipline.io/appendix.html?highlight=custom%20factor#zipline.pipeline.CustomFactor) 类。[这是 Returns 的源代码](https://www.zipline.io/_modules/zipline/pipeline/factors/basic.html#Returns)，[这是 CustomFactor 的源代码](https://www.zipline.io/_modules/zipline/pipeline/factors/factor.html#CustomFactor)。

**请打开文档和源代码链接，然后跟随我们的代码备注阅读文档和源代码**

### Inputs 变量
CustomFactor 将 `inputs` 作为类构造函数的参数，否则它会寻找类级变量 `inputs`。`inputs` 接受一个 BoundColumn 实例列表。这些实例有助于我们选择将哪种量价数据作为输入。`Returns` 类将其设为

In [ ]:
inputs = [USEquityPricing.close]

### USEquityPricing 类
[USEquityPricing](https://www.zipline.io/appendix.html?highlight=usequitypricing#zipline.pipeline.data.USEquityPricing) 类具有几个 BoundColumn 实例供我们选择。
close = USEquityPricing.close  
high = USEquityPricing.high  
low = USEquityPricing.low  
open = USEquityPricing.open  
volume = USEquityPricing.volume  

## 小测验 1
如果我们想计算收盘到开盘收益率，你认为我们应该将 USEquityPricing 中的哪些列放入列表中，并设为 `inputs`？

## 小测验 1 答案

### window_length 变量
CustomFactor 类将 `window_length`（整型）作为构造函数参数，否则寻找类级变量 `window_length`。如果我们选择 `window_length = 2`，表示我们将两天的数据（两行）传入 `compute` 函数中。

## 小测验 2
如果我们要计算每日收盘到开盘收益率，应该选择多大的窗口长度？假设我们有每日数据。

## 答案 2


### Compute 函数
`Returns` 类的函数定义包含 `compute` 函数

In [ ]:
def compute(self, today, assets, out, close):
        out[:] = (close[-1] - close[0]) / close[0]
        

* `today`：它由父类处理；它包含给定数据子集的“today”行的日期时间。在此次函数实现代码中我们不会使用它。
* `assets`：它由父类处理；它包含“out”和“close”的列标题名称。在此次函数实现代码中我们不会使用它。
* `out`：它指向一个存储计算结果的 numpy 数组。它存储的是 `compute` 函数的“return”值，而不是明确返回一个变量。
* `*input`：一个 numpy 数组元组，其中包含用来计算信号的输入数据。在 `compute` 的 `Returns` 定义中，输入是一个值 `close`，但是如果我们计算收益率需要其他数据列，可以列出更多列。


如果我们设置 `window_length=2`，那么 `compute` 函数会从 `close` 中获取两行数据。索引 1 的值是最近的值，索引 0 的值是最早的值。在 Python 中，索引 -1 表示获取索引最大的值，所以在长度为 2 的 numpy 数组中，-1 表示索引 1 处的值。

代码使用收盘价计算一天收益率，并存储到 `out` 变量中。

$ Return = \frac{close_1 - close_0}{close_0} $ 

## 小测验 3
给定开盘价 numpy 数组 `open` 和收盘价 numpy 数组 `close`，获得最近开盘价的代码是什么？假设有 2 天的数据。

## 答案 3


## 收盘到开盘收益率（隔夜收益率）

收盘到开盘收益率是指某一天收盘价与第二天开盘价的差异，公式为

$ CloseToOpen = \frac{open_1 - close_0}{close_0}$ 

下面我们将创建一个继承 `Return` 的 `CTO` 类，并覆盖 `compute` 函数。

## 小测验 4
创建一个自定义 `CloseToOpenReturns` 类，它继承自 Returns 类。定义计算隔夜收益率的 compute 函数。

In [ ]:
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import Returns

In [ ]:
class CloseToOpenReturns(Returns):
    """

    """
    
    # TODO: Set window_length (we're calculating daily returns)
    window_length = # ...

    # TODO: set inputs
    inputs = #[ ..., ...]
    
    # The compute method is passed the current day, the assets list, a pre-allocated out vector, and the
    # factor's items in the list `inputs`
    def compute(self, today, assets, out, opens, closes):
        #TODO: calculate close-to-open return and save into out[:]
        out[:] = # ...

## 后续隔夜收益率

一周的累积隔夜收益率也许可以预测未来收益率；所以是一种动量信号。

$ TrailingOvernightReturns = \sum_{1}^{Days}CloseToOpen_t$  
如果时间窗口是一周，$Days$ 可以是 5。

我们将 `CloseToOpenReturns` 当做另一个 `TrailingOvernightReturns` 类的输入，它也继承自 `Returns`。

### mask
注意，我们将创建另一个继承自 `Returns` 的类。`Returns` 继承自 [CustomFactor](https://www.zipline.io/appendix.html?highlight=factor#zipline.pipeline.CustomFactor)，它的构造函数具有 `mask` 参数。`mask` 参数接受 `Filter` 对象，该对象决定了要将哪些股票序列传递给 `compute` 函数。注意，当我们使用 `AverageDollarVolume` 并将其输出存储到变量 `universe` 中时，这个 `universe` 变量的类型是 `Filter`。

## 小测验 5
如果你要创建类型为 CloseToOpen 的对象，并将该对象定义为仅针对我们在此 notebook 的前面选择的股票池计算收益率，你会编写什么样的代码？

## 答案 5

## numpy.nansum
Numpy 具有一个将 NaN（不是数字）当做 0 的 `nansum` 函数。注意，默认情况下，如果我们向 numpy.nansum 提供一个二维数组，它会计算所有行和所有列的和。对我们来说，我们想计算 5 天（5 行）的和，每列都包含单支股票的每日收盘到开盘收益率。建议将矩阵（二维 numpy 数组）看做嵌套列表。这样更容易决定设为 `axis=0` 还是 `axis=1`。

In [ ]:
tmp = 
[ 
  [stock1day1, stock2day1 ]
  [stock1day2, stock2day2 ]
  ...
]

查看最外面的列表后发现，每个元素是一个代表一天数据的列表。如果我们使用 `np.nansum(tmp,axis=0)`，它会对每支股票的多天数据求和。如果将其看做二维矩阵，设为 `axis=0` 就相当于求每列的和。

如果设为 `axis=0`，会将 `nansum` 应用到最外面的列表（轴 0）上，所以结果为：

In [ ]:
[
  sum_of_stock_1, sum_of_stock_2
]

此外，如果设为 `axis=1`，会将 `nansum` 应用到嵌套在外部列表里的列表上。每个嵌套列表代表所有股票一天的数据，所以为：

In [ ]:
[
  sum_of_day_1,
  sum_of_day_2,
]

## numpy.nansum 使用示例

In [ ]:
tmp = np.array([
                 [1, 2, 3],
                 [np.nan, np.nan, np.nan],
                 [1, 1, 1]
               ])

print(f"Sum across rows and columns: numpy.nansum(tmp) \n{np.nansum(tmp)}")
print(f"Sum for each column: numpy.nansum(tmp,axis=0) \n{np.nansum(tmp,axis=0)}")
print(f"Sum for each row: numpy.nansum(tmp,axis=1) \n{np.nansum(tmp,axis=1)}")

## 小测验 6
对我们来说，我们想计算每个股票序列的和。你认为应该选择哪个轴？ 

## 答案 6


## 小测验 7
创建一个 TrailingOvernightReturns 类，它继承自 Returns，并且会计算隔夜收益率的累积每周之和。

In [ ]:
class TrailingOvernightReturns(Returns):
    """
    Sum of trailing close-to-open returns; we expect sentiment persistence at short horizons, so we
    look at the 5-day (ie., 1 week) window
    """
    # TODO: choose a window_length to calculate a weekly return
    window_length = # ...
    
    # TODO: set inputs to a list containing the daily close to open returns
    # Filter the close to open returns by our stock universe
    inputs = #[...]
    
    def compute(self, today, assets, out, close_to_open):
        #TODO: calculate the sum of close_to_open
        #choose the axis so that there is a sum for each stock (each column)
        #treat NaN as zeros
        out[:] = # ...

## 小测验 8
通过实例化刚刚定义的 TrailingOvernightReturns 类创建一个因子。按行业去均值，然后排名并转换为 z 分数。

In [ ]:
# TODO: create an overnight_returns_factor variable




# create a pipeline called p
p = Pipeline(screen=universe)
p.add(overnight_returns_factor, 'Overnight_Sentiment')

## 可视化管道

In [ ]:
p.show_graph(format='png')

## 运行管道并查看因子数据

In [ ]:
df = engine.run_pipeline(p, factor_start_date, universe_end_date)

In [ ]:
df.head()

## 可视化因子收益率

这些收益率是指按照单个 alpha 因子的值设定股票权重后的理论投资组合获得的收益率。

In [ ]:
from quiz_helper import make_factor_plot

In [ ]:
make_factor_plot(df, data_portal, trading_calendar, factor_start_date, universe_end_date);

## 解答
[解答 notebook](./overnight_returns_solution.ipynb)